In [1]:
import joblib
from sklearn.ensemble  import RandomForestRegressor
import xgboost as xgb
import numpy as np
import pandas as pd 
from sklearn.model_selection  import train_test_split


# 假设这里是训练数据
data = pd.read_csv(r'C:\Users\xiaowan\Desktop\works\aqi_data.csv')  
data.dropna(inplace=True)
data.columns=['Water (Kg/m³)', 'Standard Sand (Kg/m³)', 'Cement (Kg/m³)', 'Cement Grade',
            'Metakaolin (Kg/m³)', 'Active Impurity (Kg/m³)', 'Impurity Type',
            'Limestone (Kg/m³)', 'Gypsum (Kg/m³)', 'Curing Age (days)','Mpa']
X = data.loc[:,['Water (Kg/m³)', 'Standard Sand (Kg/m³)', 'Cement (Kg/m³)', 'Cement Grade',
            'Metakaolin (Kg/m³)', 'Active Impurity (Kg/m³)', 'Impurity Type',
            'Limestone (Kg/m³)', 'Gypsum (Kg/m³)', 'Curing Age (days)']]
y = data.loc[:,['Mpa']]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y
                                                    ,test_size=0.3
                                                   )

# 训练RandomForest模型
rf_model = RandomForestRegressor(
        n_estimators=100,          # 树的数量，通常越多越好，但计算成本也更高
        max_depth=10,              # 树的最大深度，控制过拟合
        min_samples_split=2,       # 节点划分最少样本数
        min_samples_leaf=1,        # 叶子节点最少样本数
        max_features='auto',       # 寻找最佳划分时要考虑的特征数量
        random_state=666            # 随机数生成器的种子，确保结果可重复
    )
rf_model.fit(X_train,  y_train)
joblib.dump(rf_model,  'rf_model.joblib') 

# 训练XGBRegressor模型
xgb_model = xgb.XGBRegressor(
        objective='reg:squarederror',  # 目标函数，这里是平方误差
        n_estimators=100,              # 树的数量
        learning_rate=0.1,             # 学习率
        max_depth=3,                   # 树的最大深度
        min_child_weight=1,            # 叶子节点最小权重和
        subsample=0.8,                 # 子样本比例
        colsample_bytree=0.8,          # 每棵树随机采样特征的比例
        gamma=0,                       # 损失函数减少的最小值
        reg_alpha=0,                   # L1正则化项
        reg_lambda=1,                  # L2正则化项
        random_state=666                # 随机数生成器的种子
    )
xgb_model.fit(X_train,  y_train)
joblib.dump(xgb_model,  'xgb_model.joblib') 

C:\Users\xiaowan\AppData\Local\Temp\ipykernel_34516\1909067426.py:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train,  y_train)
C:\Users\xiaowan\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


['xgb_model.joblib']

In [2]:
import sys
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QLabel, QLineEdit, QPushButton,
                            QGridLayout, QVBoxLayout, QHBoxLayout, QGroupBox, QFrame)
from PyQt5.QtGui import QFont, QPalette, QColor
from PyQt5.QtCore import Qt
import joblib

class AdvancedStrengthPredictor(QMainWindow):
    def __init__(self):
        super().__init__()
        # 加载模型
        self.rf_model  = joblib.load('rf_model.joblib') 
        self.xgb_model  = joblib.load('xgb_model.joblib') 
        self.init_ui() 
        self.set_style() 

    def init_ui(self):
        # 窗口基础设置
        self.setWindowTitle('Concrete   Compressive Strength Prediction System')
        self.setGeometry(300,   300, 1000, 600)

        # 输入参数分组
        input_group = QGroupBox("Material parameter input")
        self.input_labels   = [
           'Water (Kg/m³)', 'Standard Sand (Kg/m³)', 'Cement (Kg/m³)', 'Cement Grade',
            'Metakaolin (Kg/m³)', 'Active Impurity (Kg/m³)', 'Impurity Type',
            'Limestone (Kg/m³)', 'Gypsum (Kg/m³)', 'Curing Age (days)'
        ]
        self.input_fields   = [self.create_styled_input() for _ in range(10)]

        # 输入布局
        input_layout = QGridLayout()
        for i, (label, field) in enumerate(zip(self.input_labels,   self.input_fields)): 
            input_layout.addWidget(QLabel(label),   i//2*2, i%2*2)
            input_layout.addWidget(field,   i//2*2, i%2*2+1)
        input_group.setLayout(input_layout) 

        # 输出显示区域
        output_group = QGroupBox("Predict outcomes")
        self.output_labels   = [
            ('RandomForest (MPa):', self.create_output_label()), 
            ('XGBRegressor (MPa):', self.create_output_label()), 
            ('Prediction mean (MPa):', self.create_highlight_label()) 
        ]

        # 输出布局
        output_layout = QVBoxLayout()
        for label, value in self.output_labels: 
            hbox = QHBoxLayout()
            hbox.addWidget(QLabel(label),   alignment=Qt.AlignLeft)
            hbox.addWidget(value,   alignment=Qt.AlignRight)
            output_layout.addLayout(hbox) 
        output_group.setLayout(output_layout) 

        # 按钮区域
        btn_group = QWidget()
        self.predict_btn   = self.create_styled_button('Start  forecasting', '#4CAF50', 14)
        self.clear_btn   = self.create_styled_button('  Reset input', '#f44336', 14)

        btn_layout = QHBoxLayout()
        btn_layout.addWidget(self.predict_btn) 
        btn_layout.addWidget(self.clear_btn) 
        btn_group.setLayout(btn_layout) 

        # 主界面布局
        main_widget = QWidget()
        main_layout = QVBoxLayout()
        main_layout.addWidget(input_group) 
        main_layout.addWidget(self.create_divider()) 
        main_layout.addWidget(output_group) 
        main_layout.addWidget(self.create_divider()) 
        main_layout.addWidget(btn_group) 

        main_widget.setLayout(main_layout) 
        self.setCentralWidget(main_widget) 

        # 信号连接
        self.predict_btn.clicked.connect(self.predict_strength) 
        self.clear_btn.clicked.connect(self.clear_inputs) 

    def set_style(self):
        # 高级样式设置 [6]()
        self.setStyleSheet(""" 
            QGroupBox {{
                font: bold 14px '微软雅黑';
                border: 2px solid #607D8B;
                border-radius: 5px;
                margin-top: 10px;
                padding-top: 15px;
            }}
            QLabel {{
                font: 13px 'Segoe UI';
                color: #455A64;
            }}
            QLineEdit {{
                border: 1px solid #B0BEC5;
                border-radius: 3px;
                padding: 5px;
                min-width: 120px;
            }}
        """)

    # 以下为自定义样式组件创建方法
    def create_styled_input(self):
        field = QLineEdit()
        field.setStyleSheet(""" 
            QLineEdit {{
                background: #F5F5F5;
                font: 13px 'Consolas';
            }}
            QLineEdit:focus {{
                border: 2px solid #2196F3;
            }}
        """)
        return field

    def create_output_label(self):
        label = QLabel('--')
        label.setStyleSheet(""" 
            QLabel {{
                font: bold 14px 'Consolas';
                color: #37474F;
                min-width: 100px;
                padding: 5px;
                border: 1px solid #B0BEC5;
                border-radius: 3px;
            }}
        """)
        return label

    def create_highlight_label(self):
        label = QLabel('--')
        label.setStyleSheet(""" 
            QLabel {{
                font: bold 16px 'Consolas';
                color: #FFFFFF;
                background: #2196F3;
                padding: 8px;
                border-radius: 4px;
            }}
        """)
        return label

    def create_styled_button(self, text, color, size):
        btn = QPushButton(text)
        btn.setStyleSheet(f""" 
            QPushButton {{
                background: {color};
                color: white;
                font: bold {size}px '微软雅黑';
                padding: 8px 20px;
                border-radius: 4px;
                min-width: 120px;
            }}
            QPushButton:hover {{
                background: {color}DD;
            }}
        """)
        return btn

    def create_divider(self):
        line = QFrame()
        line.setFrameShape(QFrame.HLine) 
        line.setFrameShadow(QFrame.Sunken) 
        line.setStyleSheet("border:   1px solid #CFD8DC;")
        return line

    def predict_strength(self):
        try:
            # 获取输入数据
            input_data = [float(f.text())  for f in self.input_fields[:-1]]  + [int(self.input_fields[-1].text())] 
            input_data = np.array(input_data).reshape(1,  -1)

            # 调用模型进行预测
            rf_pred = self.rf_model.predict(input_data)[0] 
            xgb_pred = self.xgb_model.predict(input_data)[0] 
            avg_pred = (rf_pred + xgb_pred) / 2

            # 更新UI界面显示预测结果
            self.output_labels[0][1].setText(f"{rf_pred:.2f}") 
            self.output_labels[1][1].setText(f"{xgb_pred:.2f}") 
            self.output_labels[2][1].setText(f"{avg_pred:.2f}") 

        except Exception as e:
            self.output_labels[2][1].setText(" 输入格式错误")

    def clear_inputs(self):
        for field in self.input_fields: 
            field.clear() 
        for label in self.output_labels: 
            label[1].setText('--')

if __name__ == '__main__':
    app = QApplication(sys.argv) 
    window = AdvancedStrengthPredictor()
    window.show() 
    sys.exit(app.exec_()) 

C:\Users\xiaowan\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\xiaowan\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


SystemExit: 0

F:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
